<a href="https://colab.research.google.com/github/Imadamrii/Sentiment-_Analysis_Stocks/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk pandas


In [ ]:
import requests
import urllib.request
import time
from datetime import date
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
api_key = 'LE53BVH2ZNDQJJI8'
# Function to get news sentiment data
def get_news_sentiment(ticker):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&limit=1&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    return data

get_news_sentiment("AAPL")

{'items': '50',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'When Will the Activists Feast?',
   'url': 'https://moneymorning.com/2023/10/27/when-will-the-activists-feast/',
   'time_published': '20231027T152556',
   'authors': ['Garrett Baldwin'],
   'summary': "This is the Final Postcards from the Florida Republic to run at Money Map Press. We are moving over to Substack starting on Monday. If you're a subscriber, you don't need to do anything. But please note that you can easily white label and receive your copy starting all weekend - right here .",
   'banner_image': 'https://moneymorning.com/wp-content/blogs.dir/1/files/2023/10/PostcardOct27-207x300.png',
   'source': 'Money Morning',
   'category_within_source': 'RSS',
   'source_domain':

In [ ]:
api_key= 'O7UZDLZIQ490OIJ3'

In [12]:
def analyze_sentiment(article):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(article)
    if sentiment_score['compound'] >= 0.05:
        return 'Positive', sentiment_score
    elif sentiment_score['compound'] <= -0.05:
        return 'Negative', sentiment_score
    else:
        return 'Neutral', sentiment_score['compound']


In [13]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_market_news_sentiment(tickers, api_key, days=7):
    # Calculate the date range
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)

    # Format dates in YYYYMMDDTHHMM format
    time_to = end_date.strftime('%Y%m%dT%H%M')
    time_from = start_date.strftime('%Y%m%dT%H%M')

    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['ticker','title', 'sentiment', 'summary', 'date', 'Link to article'])

    for ticker in tickers:
        url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&time_from={time_from}&time_to={time_to}&apikey={api_key}'
        response = requests.get(url)
        data = response.json()

        # Check if 'feed' key exists in the response data
        if 'feed' not in data:
            print(f"No 'feed' key found in response for ticker {ticker}. Response data: {data}")
            continue

        articles_data = []
        for article in data['feed']:
            # Convert time_published to datetime object
            date = datetime.strptime(article['time_published'], '%Y%m%dT%H%M%S')
            # Convert datetime object to date
            date = date.date()

            # Append data to list
            articles_data.append({'ticker': ticker,
                                  'title': article['title'],
                                  'sentiment': article['overall_sentiment_label'],
                                  'summary' : article['summary'],
                                  'date': date,
                                  'Link to article': article['url']})

        # Convert list of dicts to DataFrame and concatenate
        articles_df = pd.DataFrame(articles_data)
        df = pd.concat([df, articles_df], ignore_index=True)
        df['article'] = df.apply(lambda row: row['title'] + ' ' + row['summary'], axis=1)
        df['nltk_sentiment'] = df['article'].apply(analyze_sentiment)

    return df



In [14]:
tickers = ['AAPL', 'MSFT']
days = 7
df = get_market_news_sentiment(tickers, api_key, days)



In [16]:
print(df.title.iloc[0])
sentiment_distribution = df['sentiment'].value_counts(normalize=True) * 100
print(sentiment_distribution)
print(df[['sentiment','nltk_sentiment']])
df.to_csv('output.csv', index=False, encoding='utf-8-sig', sep=';')

When Will the Activists Feast?
Somewhat-Bullish    55.0
Neutral             39.0
Bullish              5.0
Somewhat-Bearish     1.0
Name: sentiment, dtype: float64
           sentiment                                     nltk_sentiment
0            Neutral  (Positive, {'neg': 0.0, 'neu': 0.899, 'pos': 0...
1   Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.852, 'pos': 0...
2            Neutral  (Negative, {'neg': 0.138, 'neu': 0.813, 'pos':...
3            Neutral  (Positive, {'neg': 0.0, 'neu': 0.832, 'pos': 0...
4            Neutral  (Positive, {'neg': 0.0, 'neu': 0.938, 'pos': 0...
..               ...                                                ...
95  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.63, 'pos': 0....
96  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.96, 'pos': 0....
97  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.797, 'pos': 0...
98           Neutral  (Positive, {'neg': 0.053, 'neu': 0.811, 'pos':...
99  Somewhat-Bullish                         

In [ ]:
class sent_analysis:
  def _init_(self,stock_ticker):
